In [ ]:
import pandas as pd
import sqlalchemy
import pyodbc
import numpy as np
import os
from config import *

In [ ]:
from sqlalchemy.engine import URL
connection_url = URL.create("mssql+pyodbc", query={"odbc_connect": pna_connection_string})

from sqlalchemy import create_engine
engine = create_engine(connection_url)

from sqlalchemy.types import NVARCHAR
cursor = engine.connect()

In [ ]:
# Set Brand Initials: CT, KP, or WC
BRAND = 'CT'
# Set Source Code Prefix: B = CT, C = KP, D = WC
SOURCE = 'B'
# Set Year: Last 2 digits of year.
YEAR = '24'
# Set Mailing: 0-8
CATALOG = '3'
# Month Initials: 2 letter month, just for file naming conventions..
MONTH = 'MY'

In [ ]:
# Runs a sproc that generates mailing lists and generates segments based on account characteristics. 
# Don't edit the sproc to hypertune segment, segments already start at minimum acceptable values.
query = ("EXEC Netsuite.report.catalog @Brand = '"+BRAND+"'")

# Run Query.
df = pd.read_sql(query, engine)

#Create full sourcecode before processing begins.
df['sourcecode'] = SOURCE + YEAR + CATALOG + df['segment'].astype(str)

In [ ]:
#Initial Counts for review.
pd.pivot_table(df, values='customerid', index='sourcecode', aggfunc='count', margins=True, margins_name='Total')

In [ ]:
# New Customer, Domestic: 1 Purchase, LTV > $15, last Order Date < 9 months.
df_100N = df[df['segment'] == "100"]
df_100N = df_100N.sample(frac = 0.10)
df['sourcecode'] = np.where(df['customerid'].isin(df_100N['customerid']), df['sourcecode']+"N", df['sourcecode'])

In [ ]:
# New Customer, Canada: 1 Purchase, LTV > $15, last Order Date < 9 months.
df_400N = df[df['segment'] == "400"]
df_400N = df_400N.sample(frac = 0.10)
df['sourcecode'] = np.where(df['customerid'].isin(df_400N['customerid']), df['sourcecode']+"N", df['sourcecode'])

In [ ]:
# Active High Value, Domestic: Purchase > 1, LTV > $300, last Order Date < 9 months.
df_101N = df[df['segment'] == "101"]
df_101N = df_101N.sample(frac = 0.05)
df['sourcecode'] = np.where(df['customerid'].isin(df_101N['customerid']), df['sourcecode']+"N", df['sourcecode'])

In [ ]:
# Active High Value, Canada: Purchase > 1, LTV > $300, last Order Date < 9 months.
df_401N = df[df['segment'] == "401"]
df_401N = df_401N.sample(frac = 0.05)
df['sourcecode'] = np.where(df['customerid'].isin(df_401N['customerid']), df['sourcecode']+"N", df['sourcecode'])

In [ ]:
# Active, Domestic: Purchase > 1, $15 < LTV < $300, last Order Date < 9 months.
df_103N = df[df['segment'] == "103"]
df_103N = df_103N.sample(frac = 0.10)
df['sourcecode'] = np.where(df['customerid'].isin(df_103N['customerid']), df['sourcecode']+"N", df['sourcecode'])

In [ ]:
# Active, Canada: Purchase > 1, $15 < LTV < $300, last Order Date < 9 months.
df_403N = df[df['segment'] == "403"]
df_403N = df_403N.sample(frac = 0.10)
df['sourcecode'] = np.where(df['customerid'].isin(df_403N['customerid']), df['sourcecode']+"N", df['sourcecode'])

In [ ]:
# At Risk, Domestic: Purchase > 1, $15 < LTV < $300, last Order Date between 9 month and 12 months.
df_104N = df[df['segment'] == "104"]
df_104N = df_104N.sample(frac = 0.10)
df['sourcecode'] = np.where(df['customerid'].isin(df_104N['customerid']), df['sourcecode']+"N", df['sourcecode'])

In [ ]:
# At Risk, Canada: Purchase > 1, $15 < LTV < $300, last Order Date between 9 month and 12 months.
df_404N = df[df['segment'] == "404"]
df_404N = df_404N.sample(frac = 0.10)
df['sourcecode'] = np.where(df['customerid'].isin(df_404N['customerid']), df['sourcecode']+"N", df['sourcecode'])

In [ ]:
# Reactivate High Value, Domestic: Purchase > 1, LTV > $60, last Order Date between 1 year and 2 years.
df_105N = df[df['segment'] == "105"]
df_105N = df_105N.sample(frac = 0.50)
df['sourcecode'] = np.where(df['customerid'].isin(df_105N['customerid']), df['sourcecode']+"N", df['sourcecode'])

In [ ]:
# Reactivate High Value, Canada: Purchase > 1, LTV > $60, last Order Date between 1 year and 2 years.
df_405N = df[df['segment'] == "405"]
df_405N = df_405N.sample(frac = 0.50)
df['sourcecode'] = np.where(df['customerid'].isin(df_405N['customerid']), df['sourcecode']+"N", df['sourcecode'])

In [ ]:
# Old High Value, Domestic: Purchase > 1, LTV > $100, last Order Date between 2 year and 4 years.
# Look for if average days between orderse is 4x average.
df_106N = df[(df['segment'] == "106") & ((df['daystochurn'] * 4) > df['lastpurchdays'])]
df['sourcecode'] = np.where(df['customerid'].isin(df_106N['customerid']), df['sourcecode']+"N", df['sourcecode'])

In [ ]:
# Old High Value, Canada: Purchase > 1, LTV > $100, last Order Date between 2 year and 4 years.
# Look for if average days between orderse is 4x average.
df_406N = df[(df['segment'] == "406") & ((df['daystochurn'] * 4) > df['lastpurchdays'])]
df['sourcecode'] = np.where(df['customerid'].isin(df_406N['customerid']), df['sourcecode']+"N", df['sourcecode'])

In [ ]:
#Initial Counts for review.
pd.pivot_table(df, values='customerid', index='sourcecode', aggfunc='count', margins=True, margins_name='Total')

In [ ]:
# Remove extraneous columns.
columns_to_drop = ['ok_catalog', 'lifetimeorders', 'lifetimerevenue', 'catalogsends', 'catalogorders', 'brand', 'status', 'daystochurn', 'lifetime', 'lastpurchdays']
df.drop(columns=columns_to_drop, inplace=True)

In [ ]:
US_Segments = ['100', '101', '103', '104', '105', '106', '200', '000']
US_Mailing_List = df[df['segment'].isin(US_Segments) & ~df['sourcecode'].str.endswith('N') & ~df['sourcecode'].str.endswith('X')]

In [ ]:
CA_Segments = ['400', '401', '403', '404', '405', '406', '210', '010']
CA_Mailing_List = df[df['segment'].isin(CA_Segments) & ~df['sourcecode'].str.endswith('N') & ~df['sourcecode'].str.endswith('X')]

In [ ]:
All_Segments = ['100', '101', '103', '104', '105', '106', '000', '400', '401', '403', '404', '405', '406', '010']
Holdouts = df[df['segment'].isin(All_Segments) & df['sourcecode'].str.endswith('N') | df['sourcecode'].str.endswith('X')]

In [ ]:
# Create US mail files, put in applicable folders.
folder_path = os.path.join('Mailing Lists', BRAND, MONTH + YEAR)
if not os.path.exists(folder_path):
    os.makedirs(folder_path)
file_path = os.path.join(folder_path, f'{BRAND}_{MONTH}{YEAR}.csv')

US_Mailing_List.to_csv(file_path, index=False)

In [ ]:
# Create CA mail files, put in applicable folders.
folder_path = os.path.join('Mailing Lists', BRAND, MONTH + YEAR)
if not os.path.exists(folder_path):
    os.makedirs(folder_path)
file_path = os.path.join(folder_path, f'{BRAND}_{MONTH}{YEAR}_CA.csv')

CA_Mailing_List.to_csv(file_path, index=False)

In [ ]:
# Create holdout group file, put in applicable folders.
folder_path = os.path.join('Mailing Lists', BRAND, MONTH + YEAR)
if not os.path.exists(folder_path):
    os.makedirs(folder_path)
file_path = os.path.join(folder_path, f'{BRAND}_{MONTH}{YEAR}_HOLDS.csv')

Holdouts.to_csv(file_path, index=False)